# Load Data
This first section will first upload files and then load data from the server.

In [1]:

from google.colab import files
uploaded = files.upload()
!pip install tqdm

Saving train.json.zip to train.json.zip
    100% |████████████████████████████████| 51kB 1.2MB/s 
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!unzip train.json.zip

Archive:  train.json.zip
  inflating: train.json              


In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import io
import os
import sys
import json
import urllib3
import multiprocessing

from PIL import Image
from tqdm import tqdm
from urllib3.util import Retry

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


def download_image(fnames_and_urls):
    """
    download image and save its with 90% quality as JPG format
    skip image downloading if image already exists at given path
    :param fnames_and_urls: tuple containing absolute path and url of image
    """
    fname, url = fnames_and_urls
    if not os.path.exists(fname):
        http = urllib3.PoolManager(retries=Retry(connect=3, read=2, redirect=3))
        response = http.request("GET", url)
        image = Image.open(io.BytesIO(response.data))
        image_rgb = image.convert("RGB")
        image_rgb.save(fname, format='JPEG', quality=100)


def parse_dataset(_dataset, _outdir, _max=3000):
    """
    parse the dataset to create a list of tuple containing absolute path and url of image
    :param _dataset: dataset to parse
    :param _outdir: output directory where data will be saved
    :param _max: maximum images to download (change to download all dataset)
    :return: list of tuple containing absolute path and url of image
    """
    _fnames_urls = []
    with open(dataset, 'r') as f:
        data = json.load(f)
        for image in data["images"]:
            url = image["url"]
            fname = os.path.join(outdir, "{}.jpg".format(image["imageId"]))
            _fnames_urls.append((fname, url))
    return _fnames_urls[:_max]


if __name__ == '__main__':
    #if len(sys.argv) != 3:
    #    print("error: not enough arguments")
    #    sys.exit(0)

    # get args and create output directory
    dataset, outdir = "train.json", "train"
    if not os.path.exists(outdir):
        os.makedirs(outdir)

    # parse json dataset file
    fnames_urls = parse_dataset(dataset, outdir)

    # download data
    pool = multiprocessing.Pool(processes=12)
    with tqdm(total=len(fnames_urls)) as progress_bar:
        for _ in pool.imap_unordered(download_image, fnames_urls):
            progress_bar.update(1)

100%|██████████| 3000/3000 [01:00<00:00, 49.52it/s]


# Run training

In [0]:
import keras
import numpy as np

# Pre-trained models imports
from keras.applications import vgg16, xception

# Preprocessing imports
from keras.applications.imagenet_utils import decode_predictions
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator

# Layer imports
from keras.layers import Flatten, Dense
from keras import Model

# Files
from os import listdir
from os.path import isfile, join
import json
from itertools import chain, repeat, cycle

# Multi-hot encoding
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Number of classes
from tqdm import tqdm

num_classes = 210
batch_size = 128

# Path
path = './train/'

# Train labels
print("Opening train.json")
json_data = open('train.json')
data = json.load(json_data)
y_labels = [annotation['labelId'] for annotation in data['annotations'][:4000]]
y_labels = np.array(mlb.fit_transform(y_labels))
print(y_labels.shape)

# Training augmentation configuration 
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True)

def grouper(batch_size, iterable, padvalue=None):
  g = cycle(zip(*[chain(iterable, repeat(padvalue, batch_size-1))]*batch_size))
  for batch in g:
    yield list(filter(None, batch))

def multilabel_flow(path_to_data, idg, batch_size=128, target_size=(299, 299), train_or_validation='train'):
  gen = idg.flow_from_directory(path_to_data, batch_size=batch_size, target_size=target_size, classes=[train_or_validation], shuffle=False)
  labels_generator = grouper(batch_size, gen.filenames)
  for (X_batch, _), names in zip(gen, labels_generator):
    names = [n.split('/')[-1].replace('.jpg', '') for n in names]
    names = list(map(int, names))
    targets = np.take(y_labels, names)
    print(targets)
    yield X_batch, targets
  
train_generator = multilabel_flow(
    './',
    train_datagen,
    batch_size=batch_size,
    train_or_validation='train'
)

# Setup models
base_xception_model = xception.Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
x_xception = base_xception_model.output
x_xception = Flatten()(x_xception)
x_xception = Dense(512, activation='relu')(x_xception)
predictions_xception = Dense(num_classes, activation='softmax')(x_xception)

model_xception = Model(inputs=base_xception_model.input, outputs=predictions_xception)

for layer in base_xception_model.layers[:120]:
	layer.trainable = False
for layer in base_xception_model.layers[120:]:
	layer.trainable = True

model_xception.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# model_xception.load_weights('xception_checkpoint.hdf5')
# Model checkpoint
logger_xception = ModelCheckpoint('xception_checkpoint.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')

# Train input
# X_train = [xception.preprocess_input(np.expand_dims(img_to_array(load_img(join(path, f), target_size=(299, 299))), axis=0))[0] for f in tqdm(listdir(path)) if isfile(join(path, f))]
# X_test = np.array(X_train[4000:])
# X_train = np.array(X_train[:4000])
# print(X_train.shape)


# Fit the model
model_xception.fit_generator(
    train_generator,
    steps_per_epoch=2000 // batch_size,
    epochs=20,
    callbacks=[logger_xception]    
)

#filename = '../test/3.jpg
#original = xception.preprocess_input(np.expand_dims(img_to_array(load_img(filename, target_size=(224, 224))), axis=0))

#processed_image = xception.preprocess_input(original.copy())
#predictions = xception_model.predict(processed_image)
#print(decode_predictions(predictions))

Opening train.json
(4000, 210)


In [1]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         359G  7.6G  333G   3% /
tmpfs           6.4G     0  6.4G   0% /dev
tmpfs           6.4G     0  6.4G   0% /sys/fs/cgroup
/dev/root       1.2G  537M  684M  44% /opt/bin
tmpfs           6.4G  248M  6.2G   4% /usr/lib64-nvidia
/dev/sda1       365G  9.1G  356G   3% /etc/hosts
shm              64M     0   64M   0% /dev/shm
tmpfs           6.4G     0  6.4G   0% /sys/firmware
